In [1]:
MAP_KEY = '5e8bad8d50fa1ca84ea72175e2bace34'# change this key to user key

# check how many transactions we have
import pandas as pd
url = 'https://firms.modaps.eosdis.nasa.gov/mapserver/mapkey_status/?MAP_KEY=' + MAP_KEY
try:
  df = pd.read_json(url,  typ='series')
  display(df)
except:
  # possible error, wrong MAP_KEY value, check for extra quotes, missing letters
  print ("There is an issue with the query. \nTry in your browser: %s" % url)


transaction_limit             5000
current_transactions             0
transaction_interval    10 minutes
dtype: object

In [2]:
# let's create a simple function that tells us how many transactions we have used.
# We will use this in later examples

def get_transaction_count() :
  count = 0
  try:
    df = pd.read_json(url,  typ='series')
    count = df['current_transactions']
  except:
    print ("Error in our call.")
  return count

tcount = get_transaction_count()
print ('Our current transaction count is %i' % tcount)

Our current transaction count is 0


---

## API/data_availability

---

In [3]:
da_url = 'https://firms.modaps.eosdis.nasa.gov/api/data_availability/csv/' + MAP_KEY + '/all'
df = pd.read_csv(da_url)
display(df)

,data_id,min_date,max_date
0,MODIS_NRT,2025-02-01,2025-05-14
1,MODIS_SP,2000-11-01,2025-01-31
2,VIIRS_NOAA20_NRT,2025-02-01,2025-05-14
3,VIIRS_NOAA20_SP,2018-04-01,2025-01-31
4,VIIRS_NOAA21_NRT,2024-01-17,2025-05-14
5,VIIRS_SNPP_NRT,2025-02-01,2025-05-14
6,VIIRS_SNPP_SP,2012-01-20,2025-01-31
7,LANDSAT_NRT,2022-06-20,2025-05-13
8,GOES_NRT,2022-08-09,2025-05-14
9,BA_MODIS,2000-11-01,2025-02-01


**data_id** column shows the dataset id which we will need in later queries:
- 'NRT' means this is Near Real-Time dataset but it may also includes Real Time (RT) and Ultra Real Time (URT) data [click here more info on URT/RT](https://www.earthdata.nasa.gov/data/tools/firms/faq)
- 'SP' or Standard Processing; standard data products are an internally consistent, well-calibrated record of the Earth’s geophysical properties to support science. There is a multi-month lag in this dataset availability. [more information on SP vs NRT](https://www.earthdata.nasa.gov/data/tools/firms/faq)
- BA_MODIS is for MODIS burned areas product

**min_date** and **max_date** columns provide the available date range for these datasets. Dates are based on GMT

In [4]:
# now let's see how many transactions we use by querying this end point

start_count = get_transaction_count()
pd.read_csv(da_url)
end_count = get_transaction_count()
print ('We used %i transactions.' % (end_count-start_count))

# now remember, after 10 minutes this will reset

We used 5 transactions.


---

## API/area

---

Fire detection hotspots based on area, date and sensor. For more information visit https://firms.modaps.eosdis.nasa.gov/api/area

The end point expects these parameters: [MAP_KEY], [SOURCE], [AREA_COORDINATES],[DAY_RANGE] and optionally [DATE] for historical data

**NOTE** - querying the entire world for VIIRS can return between 30,000 - 100,000+ records per day


In [5]:
# in this example let's look at VIIRS NOAA-20, entire world and the most recent day
area_url = 'https://firms.modaps.eosdis.nasa.gov/api/area/csv/' + MAP_KEY + '/VIIRS_NOAA20_NRT/world/1'
start_count = get_transaction_count()
df_area = pd.read_csv(area_url)
end_count = get_transaction_count()
print ('We used %i transactions.' % (end_count-start_count))

df_area

We used 36 transactions.


,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight
0,65.04582,56.66981,335.93,0.53,0.50,2025-05-14,7,N20,VIIRS,n,2.0NRT,273.39,4.88,D
1,59.53002,34.14586,332.14,0.39,0.36,2025-05-14,9,N20,VIIRS,n,2.0NRT,281.55,4.24,N
2,60.20011,32.11379,296.04,0.39,0.36,2025-05-14,9,N20,VIIRS,n,2.0NRT,277.36,0.71,N
3,60.20112,32.10698,301.58,0.39,0.36,2025-05-14,9,N20,VIIRS,n,2.0NRT,271.82,0.76,N
4,60.20535,32.10209,308.59,0.39,0.36,2025-05-14,9,N20,VIIRS,n,2.0NRT,274.86,1.24,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21791,51.04689,2.30151,328.64,0.33,0.55,2025-05-14,1325,N20,VIIRS,n,2.0NRT,290.41,3.25,D
21792,51.08089,-3.80604,325.94,0.55,0.42,2025-05-14,1325,N20,VIIRS,n,2.0NRT,292.56,1.39,D
21793,51.36756,6.70939,330.86,0.51,0.66,2025-05-14,1325,N20,VIIRS,n,2.0NRT,300.28,3.20,D
21794,52.15322,10.40821,349.17,0.74,0.76,2025-05-14,1325,N20,VIIRS,n,2.0NRT,297.50,10.20,D


In [6]:
# We can also focus on a smaller area ex. South Asia and get the last 3 days of records
area_url = 'https://firms.modaps.eosdis.nasa.gov/api/area/csv/' + MAP_KEY + '/VIIRS_NOAA20_NRT/54,5.5,102,40/3'
df_area = pd.read_csv(area_url)
df_area

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight
0,26.57296,101.66747,334.82,0.55,0.68,2025-05-12,527,N20,VIIRS,n,2.0NRT,299.18,5.65,D
1,26.57314,101.67149,330.69,0.55,0.68,2025-05-12,527,N20,VIIRS,n,2.0NRT,299.11,4.12,D
2,26.57936,101.67035,331.82,0.55,0.68,2025-05-12,527,N20,VIIRS,n,2.0NRT,299.39,4.12,D
3,11.92029,79.11612,331.53,0.74,0.76,2025-05-12,705,N20,VIIRS,n,2.0NRT,292.30,8.14,D
4,11.92136,79.11693,334.61,0.74,0.76,2025-05-12,705,N20,VIIRS,n,2.0NRT,292.41,5.09,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14714,27.16710,56.05865,343.60,0.56,0.51,2025-05-14,954,N20,VIIRS,l,2.0NRT,318.37,22.47,D
14715,27.20523,56.21014,346.29,0.57,0.52,2025-05-14,954,N20,VIIRS,l,2.0NRT,319.23,19.80,D
14716,30.23455,56.90998,344.59,0.34,0.56,2025-05-14,954,N20,VIIRS,l,2.0NRT,319.47,6.71,D
14717,38.12440,58.42167,351.97,0.49,0.65,2025-05-14,956,N20,VIIRS,n,2.0NRT,311.04,9.48,D


List of supported countries and their 3-letter codes. This may be easier to read in html format https://firms.modaps.eosdis.nasa.gov/api/countries/?format=html however, you won't be able to see the exent box defined for each country.

Example below shows how you can view from Python.

In [7]:
# We can also focus on smaller area ex. South Asia and get last 3 days of records
countries_url = 'https://firms.modaps.eosdis.nasa.gov/api/countries'
df_countries = pd.read_csv(countries_url, sep=';')
df_countries

,id,abreviation,name,extent
0,1,ABW,Aruba,"BOX(-70.0624080069999 12.417669989,-69.8768204..."
1,2,AFG,Afghanistan,"BOX(60.4867777910001 29.3866053260001,74.89230..."
2,3,AGO,Angola,"BOX(11.6693941430001 -18.0314047239998,24.0617..."
3,4,AIA,Anguilla,"BOX(-63.4288223949999 18.1690941430001,-62.972..."
4,6,ALA,Aland Islands,"BOX(19.5131942070001 59.9044863950001,21.09669..."
...,...,...,...,...
239,234,WSM,Samoa,"BOX(-172.782582161 -14.052829685,-171.43769283..."
240,235,YEM,Yemen,"BOX(42.5457462900001 12.1114436720001,54.54029..."
241,236,ZAF,South Africa,"BOX(16.4699813160001 -46.965752863,37.97779381..."
242,237,ZMB,Zambia,"BOX(21.9798775630001 -18.0692318719999,33.6742..."


---

## API/country

---

In [8]:
# Let's see last four days MODIS data for Thailand
thai_url = 'https://firms.modaps.eosdis.nasa.gov/api/country/csv/' + MAP_KEY + '/MODIS_NRT/THA/10'
df_thai = pd.read_csv(thai_url)
df_thai

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
0,THA,14.57058,100.36417,322.98,1.23,1.10,2025-05-05,758,Aqua,MODIS,70,6.1NRT,300.91,10.60,D
1,THA,15.84613,101.76521,324.66,1.58,1.24,2025-05-05,758,Aqua,MODIS,54,6.1NRT,296.09,23.61,D
2,THA,15.91825,100.12436,327.30,1.24,1.10,2025-05-05,758,Aqua,MODIS,76,6.1NRT,297.29,18.25,D
3,THA,16.42143,100.44480,336.61,1.31,1.14,2025-05-05,758,Aqua,MODIS,86,6.1NRT,302.08,29.67,D
4,THA,16.74172,98.79063,317.03,1.09,1.04,2025-05-05,758,Aqua,MODIS,26,6.1NRT,295.75,7.66,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,THA,17.06495,100.11895,319.07,1.29,1.13,2025-05-13,238,Terra,MODIS,45,6.1NRT,289.20,13.49,D
225,THA,18.63666,100.85337,319.07,1.23,1.10,2025-05-13,238,Terra,MODIS,71,6.1NRT,296.65,8.94,D
226,THA,17.06307,100.11488,316.14,2.50,1.52,2025-05-13,818,Aqua,MODIS,57,6.1NRT,290.53,27.83,D
227,THA,17.06839,100.11929,316.02,2.50,1.52,2025-05-13,818,Aqua,MODIS,57,6.1NRT,291.01,27.26,D


## Change data format "acq_time"

In [9]:
import datetime

# Ensure "acq_time" is a string, then pad with zeros
df_thai["acq_time"] = df_thai["acq_time"].astype(str).str.zfill(4)

# Convert to HH:MM format
df_thai["acq_time"] = pd.to_datetime(df_thai["acq_time"], format="%H%M").dt.time
# df_thai["acq_time"] = df_thai["acq_time"].apply(lambda x: datetime.datetime.strptime(x, "%H:%M:%S").time())

# Check the result
print(df_thai.dtypes)
print(df_thai["acq_time"].head())  # Display first few converted times

country_id     object
latitude      float64
longitude     float64
brightness    float64
scan          float64
track         float64
acq_date       object
acq_time       object
satellite      object
instrument     object
confidence      int64
version        object
bright_t31    float64
frp           float64
daynight       object
dtype: object
0    07:58:00
1    07:58:00
2    07:58:00
3    07:58:00
4    07:58:00
Name: acq_time, dtype: object


In [10]:
unique_values = df_thai["acq_time"].unique()
print(unique_values)

[datetime.time(7, 58) datetime.time(8, 1) datetime.time(14, 46)
 datetime.time(14, 48) datetime.time(20, 12) datetime.time(3, 0)
 datetime.time(3, 2) datetime.time(7, 1) datetime.time(7, 39)
 datetime.time(7, 41) datetime.time(14, 27) datetime.time(14, 29)
 datetime.time(19, 49) datetime.time(2, 38) datetime.time(2, 41)
 datetime.time(8, 20) datetime.time(3, 19) datetime.time(7, 20)
 datetime.time(14, 10) datetime.time(14, 12) datetime.time(7, 37)
 datetime.time(8, 18) datetime.time(7, 16)]


In [11]:
df_thai.head(100)

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
0,THA,14.57058,100.36417,322.98,1.23,1.10,2025-05-05,07:58:00,Aqua,MODIS,70,6.1NRT,300.91,10.60,D
1,THA,15.84613,101.76521,324.66,1.58,1.24,2025-05-05,07:58:00,Aqua,MODIS,54,6.1NRT,296.09,23.61,D
2,THA,15.91825,100.12436,327.30,1.24,1.10,2025-05-05,07:58:00,Aqua,MODIS,76,6.1NRT,297.29,18.25,D
3,THA,16.42143,100.44480,336.61,1.31,1.14,2025-05-05,07:58:00,Aqua,MODIS,86,6.1NRT,302.08,29.67,D
4,THA,16.74172,98.79063,317.03,1.09,1.04,2025-05-05,07:58:00,Aqua,MODIS,26,6.1NRT,295.75,7.66,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,THA,18.12394,100.57619,329.11,1.04,1.02,2025-05-07,07:39:00,Aqua,MODIS,81,6.1NRT,301.85,19.31,D
96,THA,18.16073,100.69691,317.96,1.03,1.02,2025-05-07,07:39:00,Aqua,MODIS,66,6.1NRT,298.24,8.45,D
97,THA,18.16984,99.92122,319.49,1.10,1.04,2025-05-07,07:39:00,Aqua,MODIS,44,6.1NRT,293.35,10.73,D
98,THA,18.17606,99.30859,319.07,1.17,1.08,2025-05-07,07:39:00,Aqua,MODIS,64,6.1NRT,298.36,10.73,D


## Hive partition

In [12]:
import numpy as np
import pandas as pd

In [13]:
# Extract components from the "acq_date" and "acq_time" columns
df_thai["acq_year"] = pd.to_datetime(df_thai["acq_date"]).dt.year
df_thai["acq_month"] = pd.to_datetime(df_thai["acq_date"]).dt.month
df_thai["acq_day"] = pd.to_datetime(df_thai["acq_date"]).dt.day
df_thai["acq_hour"] = df_thai["acq_time"].apply(lambda x: x.hour)
df_thai["acq_minute"] = df_thai["acq_time"].apply(lambda x: x.minute)

# Write DataFrame to a directory "output_parquet" partitioned by retrieval_time
df_thai.to_parquet(
    "df_thai",
    partition_cols=["acq_year", "acq_month", "acq_day", "acq_hour", "acq_minute"],
    engine="pyarrow",
    index=False
)